In [5]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])

60000 train samples
10000 test samples
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [=

In [0]:
from keras import backend as K

def get_nth_layer_output(input_data, n):
    get_layer_output = K.function([model.layers[0].input],
                                    [model.layers[n].output])
    layer_output = get_layer_output([input_data])[0]

    return layer_output

In [0]:
train_layer_output = get_nth_layer_output(x_train, 3)
train_layer_output_last = get_nth_layer_output(x_train, 4)

test_layer_output = get_nth_layer_output(x_test, 3)
test_layer_output_last = get_nth_layer_output(x_test, 4)


In [0]:
import pandas as pd
df_train = pd.DataFrame(train_layer_output)
df2_train = pd.DataFrame(train_layer_output_last)
df_train = pd.concat([df_train, df2_train], axis=1)

df_test = pd.DataFrame(test_layer_output)
df2_test = pd.DataFrame(test_layer_output_last)
df_test = pd.concat([df_test, df2_test], axis=1)

In [0]:
(x_train_, y_train_), (x_test_, y_test_) = mnist.load_data()


In [0]:
label_train = pd.DataFrame(y_train_, columns=['target'])
label_test = pd.DataFrame(y_test_, columns=['target'])

In [11]:
!pip install catboost

     |████████████████████████████████| 63.9MB 46kB/s 


In [0]:
from catboost import CatBoostClassifier

In [0]:
model = CatBoostClassifier(iterations=3000, 
                                   verbose=10,
                                   random_seed=43,
                                   od_type='Iter',
                                   od_wait=500,
                                   train_dir=f'model_nn',
                                   task_type='GPU',
                                   )


In [0]:
df_train.columns = list(range(0, len(df_train.columns)))
df_test.columns = list(range(0, len(df_test.columns)))

In [15]:
model.fit(df_train, label_train,
                    eval_set=(df_test, label_test), 
                    use_best_model=True,
                    )

0:	learn: 2.1245753	test: 2.1272375	best: 2.1272375 (0)	total: 65.9ms	remaining: 3m 17s
10:	learn: 1.2200979	test: 1.2418470	best: 1.2418470 (10)	total: 490ms	remaining: 2m 13s
20:	learn: 0.8186812	test: 0.8486076	best: 0.8486076 (20)	total: 819ms	remaining: 1m 56s
30:	learn: 0.5811436	test: 0.6167817	best: 0.6167817 (30)	total: 1.15s	remaining: 1m 50s
40:	learn: 0.4254114	test: 0.4648545	best: 0.4648545 (40)	total: 1.5s	remaining: 1m 48s
50:	learn: 0.3168338	test: 0.3595822	best: 0.3595822 (50)	total: 1.85s	remaining: 1m 46s
60:	learn: 0.2397645	test: 0.2852355	best: 0.2852355 (60)	total: 2.2s	remaining: 1m 45s
70:	learn: 0.1834396	test: 0.2312315	best: 0.2312315 (70)	total: 2.55s	remaining: 1m 45s
80:	learn: 0.1412935	test: 0.1910785	best: 0.1910785 (80)	total: 2.89s	remaining: 1m 44s
90:	learn: 0.1097638	test: 0.1612249	best: 0.1612249 (90)	total: 3.26s	remaining: 1m 44s
100:	learn: 0.0860549	test: 0.1389403	best: 0.1389403 (100)	total: 3.62s	remaining: 1m 43s
110:	learn: 0.0678793	